In [106]:
from collections import Counter

#functions

# buy product a -> x get one for y % of the price (examples: 2 and the third would be 50%, 1 and the second would be 100%)
# CEO: GR1 + GR1 = 3.11 + 0
# COO: SR1 + SR1 + SR1 = 4.50 + 4.50 + 4.50
# VP: 11.23 + 11.23 + 11.23 = 11.23 x 3/2 + 11.23 x 3/2 + 11.23 x 3/2

# Three cases: 
# case 1: after x amount get y for free.
# case 2: after x amount each price changes to given price y.
# case 3: after x amount each price changes by a friction of original price.


# case 1: Buy X, get Y free
def free_product_included(discount_dict, count_occurances, price_dict):
    """
    Case 1: After purchasing 'promo_amount', you get 'promo_free' items for free.
    Returns the final price after applying the discount, only for discounted products.
    """
    must_pay_quantities = count_occurances.copy()
    total_price = 0

    for product, details in discount_dict.items():
        if details['discount_type'] == "free_product_included":
            promo_amount = details['promo_amount']
            promo_free = details['promo_free']
            if product in count_occurances:
                quantity = count_occurances[product]
                free_items = (quantity // (promo_amount + promo_free)) * promo_free
                must_pay_quantities[product] = quantity - free_items
                total_price += round(must_pay_quantities[product] * price_dict[product],2)

    return total_price

# case 2: After X, each additional unit has a fixed price
def fixed_price_after_x(discount_dict, count_occurances, price_dict):
    """
    Case 2: After x amount, each additional unit costs a given price y.
    """
    total_price = 0

    for product, details in discount_dict.items():
        if details['discount_type'] == "fixed_price_after_x":
            promo_amount = details['promo_amount']
            new_price = details['new_price']
            if product in count_occurances and count_occurances[product] >= promo_amount:
                total_price += round(count_occurances[product] * new_price,2)  # Price for first promo_amount units
            elif product in count_occurances:
                total_price += round(count_occurances[product] * price_dict[product],2)
    
    return total_price

# case 3: After purchasing X, all units are a fraction of the original price
def fraction_price_after_x(discount_dict, count_occurances, price_dict):
    """
    Case 3: After purchasing 'promo_amount', units will cost a fraction of the original price.
    """
    total_price = 0

    for product, details in discount_dict.items():
        if (details['discount_type'] == "fraction_price_after_x") & (product in count_occurances):
            promo_amount = details['promo_amount']
            fraction = details['fraction']
            if product in count_occurances and count_occurances[product] >= promo_amount:
                total_price += round(count_occurances[product] * price_dict[product] * fraction, 2)
            elif product in count_occurances:
                total_price += round(count_occurances[product] * price_dict[product],2)

    return total_price

# case: No discount for non-discounted products
def full_price_for_non_discounted(discount_dict, count_occurances, price_dict):
    """
    Case: For products that are not on sale, calculate the total price at their full price.
    Returns the total price for the non-discounted products.
    """
    total_price = 0
    for product in count_occurances:
        if product not in discount_dict:
            total_price += round(count_occurances[product] * price_dict[product],2)

    return total_price

In [113]:
# Product prices
price_dict = {
    "GR1": 3.11,  # Green Tea
    "SR1": 5.00,  # Strawberries
    "CF1": 11.23  # Coffee
}

# Discount types and parameters for each product
discount_dict = {
    "GR1": {"discount_type": "free_product_included", "promo_amount": 1, "promo_free": 1},  # Buy-One-Get-One-Free for Green Tea
    "SR1": {"discount_type": "fixed_price_after_x", "promo_amount": 3, "new_price": 4.50},  # Price of Strawberries drops to 4.50€ after buying 3 and more
    "CF1": {"discount_type": "fraction_price_after_x", "promo_amount": 3, "fraction": 2/3}  # Coffee price drops to 2/3 after buying 3
}

# Example cart
cart = ["GR1", "GR1", "CF1","SR1","CF1","CF1"]
count_occurances = {product: cart.count(product) for product in set(cart)}

# Apply all functions
free_product_price = free_product_included(discount_dict, count_occurances, price_dict)
fixed_price_price = fixed_price_after_x(discount_dict, count_occurances, price_dict)
fraction_price = fraction_price_after_x(discount_dict, count_occurances, price_dict)
full_price = full_price_for_non_discounted(discount_dict, count_occurances, price_dict)

# Final total price
total_price = free_product_price + fixed_price_price + fraction_price + full_price
print(total_price)

30.57


In [ ]:
#unit tests